## Lesson 19 - Social Graph with TENET





### Table of Contents

* [TENET天能](#tenet)
* [Import data to MongoDB](#import_mongo)
* [Load data from MongoDB](#load_data_from_mongodb)
* [Create Keyword Node](#create_keyword_node)
* [Create HashTag Node](#create_hashTag_node)
* [Create Mention Node](#create_mention_node)
* [Create Facebook post Node](#create_facebook_post_node)
* [Create Facebook fanpage Node](#create_facebook_fanpage_node)
* [Create Facebook group Node](#create_facebook_group_node)
* [Create Facebook profile Node](#create_facebook_profile_node)
* [Create Facebook ExpandedLink Node](#create_facebook_expandedLink_node)
* [Relationship of fb page POSTS fb post](#relationship_fbpage_fbpost)
* [Relationship of fb group POSTS fb post](#relationship_fbgroup_fbpost)
* [Relationship of fb profile POSTS fb post](#relationship_fbprofile_fbpost)
* [Relationship of fb ExpandedLink and POSTS](#relationship_fb_expandedLink_posts)
* [Relationship of FacebookPost keywords and post](#relationship_keywords_and_post)
* [Relationship of FacebookPost hashtag and post](#Relationship_facebookPost_hashtag_post)
* [Relationship of Facebook mention and post](#relationship_mention_and_post)
* [Create Instagram post Node](#create_instagram_post_node)
* [Create Instagram user Node](#create_instagram_user_node)
* [Relationship of Instagram user and post](#relationship_ig_user_post)
* [Relationship of Instagram keywords and post](#relationship_ig_keywords_post)
* [Relationship of Instagram hashtag and post](#relationship_ig_hashtag_post)
* [Relationship of Instagram mention and post](#relationship_ig_mention_post)
* [Import to Neo4j](#import_neo4j)




<a id="tenet"></a>
## TENET天能

- 上映日期：2020-08-27
- 期待度 99% 網友想看 滿意度 4.1 綜合評分
- 資料區間：上映前一周2020-08-17 ~ 上映後一個月2020-09-27
- 資料來源：Facebook、Instagram

<a id="import_mongo"></a>
## Import data to MongoDB

- ./data/mongo/Tenet/

```
mongorestore -h 127.0.0.1 --port 27017 --db Tenet Tenet --drop
```

<img src="images/import_tenet_to_mongodb.png">

<img src="images/import_tenet_to_mongodb_done.png">

<img src="images/robomongo_tenet.png">

In [1]:
import csv
import requests
import json
import time
import datetime
import pandas as pd
import numpy as np

from datetime import (datetime as dt, timedelta as td)
from pymongo import MongoClient
from pymongo import UpdateOne
from urllib.parse import quote, unquote

In [2]:
import jieba
import jieba.analyse

jieba.initialize()
jieba.set_dictionary("./dict/dict.big.txt")
jieba.load_userdict('./dict/mydic.txt')

wantWordList = set()
with open('./dict/mydic.txt', 'r', encoding="utf8") as file:
    wantWordList = file.readlines()
    wantWordList = [wantword.strip('\n').replace('\ufeff','').strip().split(' ')[0] for wantword in wantWordList]

stopwords = set()
with open('./dict/stopwords.txt', 'r', encoding="utf8") as file:
    stopwords = file.readlines()
    stopwords = [stopword.strip('\n').strip() for stopword in stopwords]
except_file = open("./dict/hippo_exception_word.txt", encoding='utf-8')
exception = except_file.read().split(',')
exception.append(" ")

punct = set(u''':!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
punct |= set(exception)
punct = list(punct)
stopwords = set(stopwords+punct)

names = set()
with open('./dict/name.txt', 'r', encoding="utf-8") as file:
    names= file.readlines()
    names = [name.strip('\n').strip() for name in names]

def segmentWord(text):
    words = [word for word in jieba.cut(text, cut_all=False) if len(word.strip())>1 and (word not in stopwords)  ]
    return "/".join(set(words))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.901 seconds.
Prefix dict has been built succesfully.
Building prefix dict from C:\Programming\Neo4j\dict\dict.big.txt ...
Loading model from cache C:\Users\Admin\AppData\Local\Temp\jieba.u75d3f7fb10c9300db2253d2405c771ed.cache
Loading model cost 1.630 seconds.
Prefix dict has been built succesfully.


In [3]:
import jieba
import jieba.analyse

jieba.initialize()

jieba.set_dictionary("./dict/dict.big.txt")
jieba.load_userdict('./dict/mydic.txt')

wantWordList = set()
with open('./dict/mydic.txt', 'r', encoding="utf8") as file:
    wantWordList = file.readlines()
    wantWordList = [wantword.strip('\n').replace('\ufeff','').strip().split(' ')[0] for wantword in wantWordList]

stopwords = set()
with open('./dict/stopwords.txt', 'r', encoding="utf8") as file:
    stopwords = file.readlines()
    stopwords = [stopword.strip('\n').strip() for stopword in stopwords]
except_file = open("./dict/hippo_exception_word.txt", encoding='utf-8')
exception = except_file.read().split(',')
exception.append(" ")

punct = set(u''':!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
punct |= set(exception)
punct = list(punct)
stopwords = set(stopwords+punct)

names = set()
with open('./dict/name.txt', 'r', encoding="utf-8") as file:
    names= file.readlines()
    names = [name.strip('\n').strip() for name in names]

def segmentWord(text):
    words = [word for word in jieba.cut(text, cut_all=False) if len(word.strip())>1 and (word not in stopwords)  ]
    return "/".join(set(words))

Building prefix dict from C:\Programming\Neo4j\dict\dict.big.txt ...
Loading model from cache C:\Users\Admin\AppData\Local\Temp\jieba.u75d3f7fb10c9300db2253d2405c771ed.cache
Loading model cost 1.584 seconds.
Prefix dict has been built succesfully.


In [4]:
MONGO_URI = 'mongodb://127.0.0.1:27017'
client = MongoClient(MONGO_URI)
collection_name = 'Tenet'
db = client[collection_name]

In [5]:
def media_id_to_code(media_id):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-_'
    short_code = ''
    while media_id > 0:
        remainder = media_id % 64
        media_id = (media_id-remainder)//64
        short_code = alphabet[remainder] + short_code
    return short_code

def code_to_media_id(short_code):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-_'
    media_id = 0;
    for letter in short_code:
        media_id = (media_id*64) + alphabet.index(letter)

    return media_id

def extract_short_code(platformId):
    media_id = platformId.split('_')[0]
    return media_id_to_code(int(media_id))

def extract_content_hashtag(content):
    hashtags = []
    context = ''
    if '#' in content:
        context = content.split('#')[0].strip()
        for h in content.split('#')[1:]:
            hashtags.append('#'+h.split(' ')[0].strip())
    else:
        context = content
    return context, hashtags

def extract_context(content):
    context = ''
    if type(content)==float:
        return ""
    if '#' in content:
        context = content.split('#')[0].strip()
    else:
        context = content
    return context

def extract_hashtag(content):
    hashtags = []
    if type(content)==float:
        return ""
    if '#' in content:
        for h in content.split('#')[1:]:
            hashtags.append('#'+h.split(' ')[0].strip())
    return "/".join(hashtags) if hashtags else ""

def extract_mention(content):
    mentions = []
    if type(content)==float:
        return ""
    if '@' in content:
        for h in content.split('@')[1:]:
            mentions.append('@'+h.split(' ')[0].strip())
    return "/".join(mentions) if mentions else ""

def extract_ig_username(postUrl):
    try:
        # https://www.instagram.com/dunzo_it/
        if postUrl[-1]=='/':
            return str(postUrl).replace('https://www.instagram.com/','')[:-1]
        else:
            return str(postUrl).replace('https://www.instagram.com/','')
        
    except:
        return ""


<a id="load_data_from_mongodb"></a>
## Load data from MongoDB

In [6]:
result = list(db[collection_name].find({  }))
print(len(result))

2803


In [7]:
df = pd.DataFrame(result)
if '_id' in df.columns:
    del df['_id']

df = df.replace(np.nan, '', regex=True)
print(df.shape[0])
df.head(1)

2803


,platformId,platform,date,updated,type,description,message,expandedLinks,link,postUrl,...,statistics,account,legacyId,id,keyword,title,caption,videoLengthMS,imageText,liveVideoStatus
0,10164803303730179,Facebook,2020-09-26 22:38:25,2020-10-20 19:53:19,photo,有人話《天能》好燒腦，要睇埋電影分析先明...\n我就話閱哦嘅《低能》更加燒腦，聽埋佢解釋都...,低能。 釋懷啦!!,[{'original': 'https://www.facebook.com/Pokegu...,https://www.facebook.com/PokeguideHK/photos/a....,https://www.facebook.com/narcissistyoyo/posts/...,...,"{'actual': {'likeCount': 112, 'shareCount': 13...","{'id': 3420777, 'name': 'Gregory Wong', 'handl...",107572299356,3420777|10164803303730179,天能_AND_電影,,,,,


In [8]:
def extract_actual_likeCount(statistics):
    if 'likeCount' in statistics["actual"]:
        actual_likeCount = statistics["actual"]["likeCount"]
    else:
        actual_likeCount = 0
    return actual_likeCount

def extract_actual_commentCount(statistics):
    if 'commentCount' in statistics["actual"]:
        actual_commentCount = statistics["actual"]["commentCount"]
    else:
        actual_commentCount = 0
    return actual_commentCount

def extract_actual_shareCount(statistics):
    if 'shareCount' in statistics["actual"]:
        actual_shareCount = statistics["actual"]["shareCount"]
    else:
        actual_shareCount = 0
    return actual_shareCount

def extract_actual_loveCount(statistics):
    if 'loveCount' in statistics["actual"]:
        actual_loveCount = statistics["actual"]["loveCount"]
    else:
        actual_loveCount = 0
    return actual_loveCount

def extract_actual_wowCount(statistics):
    if 'wowCount' in statistics["actual"]:
        actual_wowCount = statistics["actual"]["wowCount"]
    else:
        actual_wowCount = 0
    return actual_wowCount

def extract_actual_hahaCount(statistics):
    if 'hahaCount' in statistics["actual"]:
        actual_hahaCount = statistics["actual"]["hahaCount"]
    else:
        actual_hahaCount = 0
    return actual_hahaCount

def extract_actual_sadCount(statistics):
    if 'sadCount' in statistics["actual"]:
        actual_sadCount = statistics["actual"]["sadCount"]
    else:
        actual_sadCount = 0
    return actual_sadCount

def extract_actual_angryCount(statistics):
    if 'angryCount' in statistics["actual"]:
        actual_angryCount = statistics["actual"]["angryCount"]
    else:
        actual_angryCount = 0
    return actual_angryCount

def extract_actual_thankfulCount(statistics):
    if 'thankfulCount' in statistics["actual"]:
        actual_thankfulCount = statistics["actual"]["thankfulCount"]
    else:
        actual_thankfulCount = 0
    return actual_thankfulCount

def extract_actual_careCount(statistics):
    if 'careCount' in statistics["actual"]:
        actual_careCount = statistics["actual"]["careCount"]
    else:
        actual_careCount = 0
    return actual_careCount

if 'statistics' in df.columns:
    df["actual_likeCount"] = df["statistics"].apply(lambda x: extract_actual_likeCount(x))
    df["actual_commentCount"] = df["statistics"].apply(lambda x: extract_actual_commentCount(x))
    df["actual_shareCount"] = df["statistics"].apply(lambda x: extract_actual_shareCount(x))
    df["actual_loveCount"] = df["statistics"].apply(lambda x: extract_actual_loveCount(x))
    df["actual_wowCount"] = df["statistics"].apply(lambda x: extract_actual_wowCount(x))
    df["actual_hahaCount"] = df["statistics"].apply(lambda x: extract_actual_hahaCount(x))
    df["actual_sadCount"] = df["statistics"].apply(lambda x: extract_actual_sadCount(x))
    df["actual_angryCount"] = df["statistics"].apply(lambda x: extract_actual_angryCount(x))
    df["actual_thankfulCount"] = df["statistics"].apply(lambda x: extract_actual_thankfulCount(x))
    df["actual_careCount"] = df["statistics"].apply(lambda x: extract_actual_careCount(x))

def extract_expected_likeCount(statistics):
    if 'likeCount' in statistics["expected"]:
        expected_likeCount = statistics["expected"]["likeCount"]
    else:
        expected_likeCount = 0
    return expected_likeCount

def extract_expected_commentCount(statistics):
    if 'commentCount' in statistics["expected"]:
        expected_commentCount = statistics["expected"]["commentCount"]
    else:
        expected_commentCount = 0
    return expected_commentCount

def extract_expected_shareCount(statistics):
    if 'likeCount' in statistics["expected"]:
        expected_shareCount = statistics["expected"]["shareCount"]
    else:
        expected_shareCount = 0
    return expected_shareCount

def extract_expected_loveCount(statistics):
    if 'loveCount' in statistics["expected"]:
        expected_loveCount = statistics["expected"]["loveCount"]
    else:
        expected_loveCount = 0
    return expected_loveCount

def extract_expected_wowCount(statistics):
    if 'wowCount' in statistics["expected"]:
        expected_wowCount = statistics["expected"]["wowCount"]
    else:
        expected_wowCount = 0
    return expected_wowCount

def extract_expected_hahaCount(statistics):
    if 'hahaCount' in statistics["expected"]:
        expected_hahaCount = statistics["expected"]["hahaCount"]
    else:
        expected_hahaCount = 0
    return expected_hahaCount

def extract_expected_sadCount(statistics):
    if 'sadCount' in statistics["expected"]:
        expected_sadCount = statistics["expected"]["sadCount"]
    else:
        expected_sadCount = 0
    return expected_sadCount

def extract_expected_angryCount(statistics):
    if 'angryCount' in statistics["expected"]:
        expected_angryCount = statistics["expected"]["angryCount"]
    else:
        expected_angryCount = 0
    return expected_angryCount

def extract_expected_thankfulCount(statistics):
    if 'thankfulCount' in statistics["expected"]:
        expected_thankfulCount = statistics["expected"]["thankfulCount"]
    else:
        expected_thankfulCount = 0
    return expected_thankfulCount

def extract_expected_careCount(statistics):
    if 'careCount' in statistics["expected"]:
        expected_careCount = statistics["expected"]["careCount"]
    else:
        expected_careCount = 0
    return expected_careCount

if 'statistics' in df.columns:
    df["expected_likeCount"] = df["statistics"].apply(lambda x: extract_expected_likeCount(x))
    df["expected_commentCount"] = df["statistics"].apply(lambda x: extract_expected_commentCount(x))
    df["expected_shareCount"] = df["statistics"].apply(lambda x: extract_expected_shareCount(x))
    df["expected_loveCount"] = df["statistics"].apply(lambda x: extract_expected_loveCount(x))
    df["expected_wowCount"] = df["statistics"].apply(lambda x: extract_expected_wowCount(x))
    df["expected_hahaCount"] = df["statistics"].apply(lambda x: extract_expected_hahaCount(x))
    df["expected_sadCount"] = df["statistics"].apply(lambda x: extract_expected_sadCount(x))
    df["expected_angryCount"] = df["statistics"].apply(lambda x: extract_expected_angryCount(x))
    df["expected_thankfulCount"] = df["statistics"].apply(lambda x: extract_expected_thankfulCount(x))
    df["expected_careCount"] = df["statistics"].apply(lambda x: extract_expected_careCount(x))

def extract_expandedLinks(expandedLinks):
    ret_links = ''
    if expandedLinks:
        for links in expandedLinks:
            if 'original' in links:
                ret_links += links['original'] +','
    return ret_links[:-1]

if 'expandedLinks' in df.columns:
    df["expandlinks"] = df["expandedLinks"].apply(lambda x: extract_expandedLinks(x))

def extract_media_type(media):
    ret = ''
    if media:
        for m in media:
            ret += m["type"]+','
    return ret[:-1]

if 'media' in df.columns:
    df["media_type"] = df["media"].apply(lambda x: extract_media_type(x))

def extract_account_id(account):
    return account["id"]

def extract_account_name(account):
    return account["name"]

def extract_account_page_url(account):
    ret = ""
    if 'handle' in account:
        if 'Facebook' in account['platform']:
            ret = "https://www.facebook.com/{}".format(account["handle"])
        elif 'Instagram' in account['platform']:
            ret = "https://www.instagram.com/{}".format(account["handle"])+'/'
        else:
            ret = account["handle"]
    return ret

def extract_account_profileImage(account):
    return account["profileImage"]

def extract_account_id_url(account):
    return account["url"]

def extract_account_fanpage_id(account):
    if 'platformId' in  account:
        return account["platformId"]
    else:
        return ''

def extract_account_subscriberCount(account):
    return account["subscriberCount"]

def extract_account_accountType(account):
    ret = ""
    if 'accountType' in account:
        ret = account["accountType"]
    return ret

def extract_account_pageAdminTopCountry(account):
    return account["pageAdminTopCountry"]

def extract_account_verified(account):
    return account["verified"]

def add_quote(keyword):
    return '"'+str(keyword).replace('\n', ' ').replace('\r', ' ').strip()+'"'

def replace_double_quote(s):
    return str(s).strip().replace('"',"'")

def if_in_list(item, target_list):
    if item not in target_list:
        return 0
    return 1

if 'account' in df.columns:
    df["page_account_id"] = df["account"].apply(lambda x: extract_account_id(x))
    df["page_name"] = df["account"].apply(lambda x: extract_account_name(x))
    df["page_url"] = df["account"].apply(lambda x: extract_account_page_url(x))
    df["page_id_url"] = df["account"].apply(lambda x: extract_account_id_url(x))
    df["page_profileImage"] = df["account"].apply(lambda x: extract_account_profileImage(x))
    df["page_subscriberCount"] = df["account"].apply(lambda x: extract_account_subscriberCount(x))
    df["page_id"] = df["account"].apply(lambda x: extract_account_fanpage_id(x))
    df["page_accountType"] = df["account"].apply(lambda x: extract_account_accountType(x))
    df["page_verified"] = df["account"].apply(lambda x: extract_account_verified(x))
df.head(1)

,platformId,platform,date,updated,type,description,message,expandedLinks,link,postUrl,...,media_type,page_account_id,page_name,page_url,page_id_url,page_profileImage,page_subscriberCount,page_id,page_accountType,page_verified
0,10164803303730179,Facebook,2020-09-26 22:38:25,2020-10-20 19:53:19,photo,有人話《天能》好燒腦，要睇埋電影分析先明...\n我就話閱哦嘅《低能》更加燒腦，聽埋佢解釋都...,低能。 釋懷啦!!,[{'original': 'https://www.facebook.com/Pokegu...,https://www.facebook.com/PokeguideHK/photos/a....,https://www.facebook.com/narcissistyoyo/posts/...,...,photo,3420777,Gregory Wong,https://www.facebook.com/narcissistyoyo,https://facebook.com/narcissistyoyo,https://scontent-sea1-1.xx.fbcdn.net/v/t1.0-1/...,26334,,facebook_profile,True


In [9]:
df_fb = df[df["platform"]=="Facebook"]
df_fb = df_fb.rename(columns={'platformId': 'status_id'})
df_fb["context"] = df_fb["title"]+df_fb["description"]+df_fb["message"]
df_fb["context"] = df_fb["message"].apply(lambda x:extract_context(x))
df_fb["hashtags"] = df_fb["message"].apply(lambda x:extract_hashtag(x))
df_fb["mention"] = df_fb["message"].apply(lambda x:extract_mention(x))
df_fb['keywords'] = list(map(segmentWord, df_fb['context']))
df_fb.fillna('', inplace=True)
print(df_fb.shape[0])

2594


In [10]:
df_fb['keywords'].sample(3)

2537                    
2342    拍電影/之後/諾蘭/阿湯哥/稍微
613                     
Name: keywords, dtype: object

In [11]:
df_fb.page_accountType.unique()

array(['facebook_profile', 'facebook_page', 'facebook_group'],
      dtype=object)

In [12]:
df_ig = df[df["platform"]=='Instagram']
df_ig["context"] = df_ig["title"]+df_ig["description"]+df_ig["message"]
df_ig["context"] = df_ig["description"].apply(lambda x:extract_context(x))
df_ig["shortCode"] = df_ig["platformId"].apply(lambda x : extract_short_code(x))
df_ig["postId"] = df_ig["shortCode"].apply(lambda x : code_to_media_id(x))
df_ig["hashtags"] = df_ig["description"].apply(lambda x:extract_hashtag(x))
df_ig["mention"] = df_ig["description"].apply(lambda x:extract_mention(x))
df_ig["userName"] = df_ig["page_id_url"].apply(lambda x : extract_ig_username(x))
df_ig['keywords'] = list(map(segmentWord, df_ig['context']))
df_ig.fillna('', inplace=True)
df_ig.sample(1)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,platformId,platform,date,updated,type,description,message,expandedLinks,link,postUrl,...,page_id,page_accountType,page_verified,context,shortCode,postId,hashtags,mention,userName,keywords
2722,2390462407752351874,Instagram,2020-09-04 02:09:07,2020-09-27 08:56:39,photo,多謝華納電影公司！天能個叉電器好靚，用咗我可唔可以回到過去呢？！😭,,,,https://www.instagram.com/p/CEsoOUHAQCC/,...,1960905462,,False,多謝華納電影公司！天能個叉電器好靚，用咗我可唔可以回到過去呢？！😭,CEsoOUHAQCC,2390462407752351874,,,wanhoi_hongkong,電影/天能/公司/好靚/電器/華納/多謝/回到過去


<a id="create_keyword_node"></a>
## Create Keyword Node

In [13]:
fb_keyword_list = list(df_fb[df_fb['keywords'].str.len() > 0]['keywords'])
ig_keyword_list = list(df_ig[df_ig['keywords'].str.len() > 0]['keywords'])
df_keyword = pd.DataFrame(fb_keyword_list+ig_keyword_list, columns=['keywords'])
bag_keyword_list = []
keyword_id_map = {}
global_bag_keyword_list = []
cnt_keyword = 100_000_000
for index, row in df_keyword.iterrows():
    for keyword in row['keywords'].split('/'):
        if keyword not in global_bag_keyword_list:
            cnt_keyword = cnt_keyword+1
            cnt_keyword_code = "K"+str(cnt_keyword)
            keyword_id_map[keyword] = cnt_keyword_code
            bag_keyword_list.append({"id":cnt_keyword_code, "keyword":keyword})
            global_bag_keyword_list.append(keyword)

In [14]:
list(keyword_id_map.keys())[:3]

['釋懷', '低能', '想見你']

In [15]:
if global_bag_keyword_list:
    # Create keyword nodes
    all_keyword_nodes = pd.DataFrame(bag_keyword_list)
    all_keyword_nodes = all_keyword_nodes.rename(columns={'id':'Keyword:ID'})
    all_keyword_nodes[":LABEL"] = "Keywords"
    all_keyword_nodes["keyword"] = all_keyword_nodes["keyword"].apply(lambda x : replace_double_quote(x))
    all_keyword_nodes["keyword"] = all_keyword_nodes["keyword"].apply(lambda x: add_quote(x))
    print(all_keyword_nodes.shape[0])
    all_keyword_nodes.to_csv('opt/csv/ct_tenet/post_keyword_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
else:
    # Create empty keyword nodes
    keyword_id_map = {}
    all_keyword_nodes = pd.DataFrame(columns=['Keyword:ID', 'keyword', ':LABEL'])
    all_keyword_nodes.to_csv('opt/csv/ct_tenet/post_keyword_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')

19782


<a id="create_hashTag_node"></a>
## Create HashTag Node

In [16]:
fb_hashtag_list = list(df_fb[df_fb['hashtags'].str.len() > 0]['hashtags'])
ig_hashtag_list = list(df_ig[df_ig['hashtags'].str.len() > 0]['hashtags'])
df_hashtag = pd.DataFrame(fb_hashtag_list+ig_hashtag_list, columns=['hashtags'])
bag_hashtags_list = []
hashtags_id_map = {}
global_bag_hashtags_list = []
cnt_hashtag = 100_000_000
for index, row in df_hashtag.iterrows():
    for hashtag in row['hashtags'].split('/'):
        if hashtag not in global_bag_hashtags_list:
            cnt_hashtag = cnt_hashtag+1
            cnt_hashtag_code = "H"+str(cnt_hashtag)
            hashtags_id_map[hashtag] = cnt_hashtag_code
            bag_hashtags_list.append({"id":cnt_hashtag_code, "hashtags":hashtag})
            global_bag_hashtags_list.append(hashtag)

In [17]:
list(hashtags_id_map.keys())[:3]

['#01周報', '#願者上網', '#社交媒體']

In [18]:
if global_bag_hashtags_list:
    # Create hashtags nodes
    all_hashtags_nodes = pd.DataFrame(bag_hashtags_list)
    all_hashtags_nodes = all_hashtags_nodes.rename(columns={'id':'Hashtag:ID'})
    all_hashtags_nodes[":LABEL"] = "Hashtag"
    all_hashtags_nodes["hashtags"] = all_hashtags_nodes["hashtags"].apply(lambda x : replace_double_quote(x))
    all_hashtags_nodes["hashtags"] = all_hashtags_nodes["hashtags"].apply(lambda x: add_quote(x))
    print(all_hashtags_nodes.shape[0])
    all_hashtags_nodes.to_csv('opt/csv/ct_tenet/post_hashtags_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
else:
    # Create empty hashtags nodes
    hashtags_id_map = {}
    all_hashtags_nodes = pd.DataFrame(columns=['Hashtag:ID', 'hashtags', ':LABEL'])
    all_hashtags_nodes.to_csv('opt/csv/ct_tenet/post_hashtags_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')

3677


In [19]:
all_hashtags_nodes.tail()

,Hashtag:ID,hashtags,:LABEL
3672,H100003673,"""#ELLEMEN""",Hashtag
3673,H100003674,"""#ELLEMENwatch""",Hashtag
3674,H100003675,"""#ELLEMENselect""",Hashtag
3675,H100003676,"""#menstyle""",Hashtag
3676,H100003677,"""#menwatch""",Hashtag


<a id="create_mention_node"></a>
## Create Mention Node

In [20]:
fb_mention_list = list(df_fb[df_fb['mention'].str.len() > 0]['mention'])
ig_mention_list = list(df_ig[df_ig['mention'].str.len() > 0]['mention'])
df_mention = pd.DataFrame(fb_mention_list+ig_mention_list, columns=['mention'])
bag_mentions_list = []
mentions_id_map = {}
global_bag_mention_list = []
cnt_mention = 100_000_000
for index, row in df_mention.iterrows():
    for mention in row['mention'].split('/'):
        if mention not in global_bag_mention_list:
            cnt_mention = cnt_mention+1
            cnt_mention_code = "M"+str(cnt_mention)
            mentions_id_map[mention] = cnt_mention_code
            bag_mentions_list.append({"id":cnt_mention_code, "mention":mention})
            global_bag_mention_list.append(mention)

In [21]:
list(mentions_id_map.keys())[:3]

['@Every', '@Group', '@groupbuyer.hk']

In [22]:
if global_bag_mention_list:
    # Create mention nodes
    all_mention_nodes = pd.DataFrame(bag_mentions_list)
    all_mention_nodes = all_mention_nodes.rename(columns={'id':'Mention:ID'})
    all_mention_nodes[":LABEL"] = "Mention"
    all_mention_nodes["mention"] = all_mention_nodes["mention"].apply(lambda x : replace_double_quote(x))
    all_mention_nodes["mention"] = all_mention_nodes["mention"].apply(lambda x: add_quote(x))
    print(all_mention_nodes.shape[0])
    all_mention_nodes.to_csv(f'opt/csv/ct_tenet/post_mention_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
else:
    # Create empty mention nodes
    mention_id_map = {}
    all_mention_nodes = pd.DataFrame(columns=['Mention:ID', 'mention', ':LABEL'])
    all_mention_nodes.to_csv('opt/csv/ct_tenet/post_mention_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')

157


<a id="create_facebook_post_node"></a>
## Create Facebook post Node

In [23]:
fb_post_nodes = df_fb[['status_id','postUrl','message','date','actual_likeCount','actual_commentCount','actual_shareCount','actual_loveCount','actual_wowCount','actual_hahaCount','actual_sadCount','actual_angryCount','actual_thankfulCount','actual_careCount','page_account_id','page_name','page_url','page_id_url']].copy()
fb_post_nodes["postId"] = fb_post_nodes["status_id"]
fb_post_nodes = fb_post_nodes.rename(columns={'status_id': 'statusId',
                                              'message': 'text',
                                              'date': 'created',
                                              'actual_likeCount': 'likeCount',
                                              'actual_commentCount': 'commentCount',
                                              'actual_shareCount': 'shareCount',
                                              'actual_loveCount': 'loveCount',
                                              'actual_wowCount': 'wowCount',
                                              'actual_hahaCount': 'hahaCount',
                                              'actual_sadCount': 'sadCount',
                                              'actual_angryCount': 'angryCount',
                                              'actual_thankfulCount': 'thankfulCount',
                                              'actual_careCount': 'careCount',
                                              'page_account_id': 'fanpageId',
                                              'page_name': 'fanpageName',
                                              'page_url': 'pageUrl',
                                              'page_id_url':'pageIdUrl'})
fb_post_nodes[":LABEL"] = "FacebookPost"
fb_post_nodes = fb_post_nodes.rename(columns={'statusId': 'FacebookPost:ID', 'text': 'text'})
fb_post_nodes = fb_post_nodes.drop_duplicates(subset='FacebookPost:ID', keep="first")
fb_post_nodes["text"] = fb_post_nodes["text"].apply(lambda x : replace_double_quote(x))
fb_post_nodes["text"] = fb_post_nodes["text"].apply(lambda x : add_quote(x))
fb_post_nodes["fanpageName"] = fb_post_nodes["fanpageName"].apply(lambda x : replace_double_quote(x))
fb_post_nodes["fanpageName"] = fb_post_nodes["fanpageName"].apply(lambda x : add_quote(x))
fb_post_nodes.to_csv('opt/csv/ct_tenet/fb_post_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_post_nodes.shape[0])
fb_post_nodes.head(1)

2594


,FacebookPost:ID,postUrl,text,created,likeCount,commentCount,shareCount,loveCount,wowCount,hahaCount,sadCount,angryCount,thankfulCount,careCount,fanpageId,fanpageName,pageUrl,pageIdUrl,postId,:LABEL
0,10164803303730179,https://www.facebook.com/narcissistyoyo/posts/...,"""低能。 釋懷啦!!""",2020-09-26 22:38:25,112,0,13,0,1,81,1,2,0,2,3420777,"""Gregory Wong""",https://www.facebook.com/narcissistyoyo,https://facebook.com/narcissistyoyo,10164803303730179,FacebookPost


<a id="create_facebook_fanpage_node"></a>
## Create Facebook fanpage Node

In [24]:
df_fb_page = df_fb[df_fb["page_accountType"]=="facebook_page"].copy()
fb_page_nodes = df_fb_page[['page_account_id','page_name','page_url','page_id_url','page_profileImage','page_subscriberCount','page_id','page_accountType','page_verified']]
fb_page_nodes = fb_page_nodes.rename(columns={'page_account_id': 'fanpageId',
                                              'page_name': 'fanpageName',
                                              'page_url': 'pageUrl',
                                              'page_id_url': 'pageIdUrl',
                                              'page_profileImage': 'profileImage',
                                              'page_subscriberCount': 'subscriberCount',
                                              'page_id': 'pageId',
                                              'page_accountType': 'accountType',
                                              'page_verified': 'verified'})
fb_page_nodes[":LABEL"] = "FacebookFanPage"
fb_page_nodes = fb_page_nodes.rename(columns={'fanpageId': 'FacebookFanPage:ID', 'fanpageName': 'fanpageName'})
fb_page_nodes = fb_page_nodes.drop_duplicates(subset='FacebookFanPage:ID', keep="first")
fb_page_nodes["fanpageName"] = fb_page_nodes["fanpageName"].apply(lambda x : replace_double_quote(x))
fb_page_nodes["fanpageName"] = fb_page_nodes["fanpageName"].apply(lambda x : add_quote(x))
fb_page_nodes.to_csv('opt/csv/ct_tenet/fb_page_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_page_nodes.shape[0])
fb_page_nodes.sample(1)

748


,FacebookFanPage:ID,fanpageName,pageUrl,pageIdUrl,profileImage,subscriberCount,pageId,accountType,verified,:LABEL
244,6867117,"""電影絮語""",https://www.facebook.com/moviechatters,https://www.facebook.com/1944031459251610,https://scontent-sea1-1.xx.fbcdn.net/v/t1.0-1/...,1783,1944031459251610,facebook_page,False,FacebookFanPage


<a id="create_facebook_group_node"></a>
## Create Facebook group Node

In [25]:
df_fb_group = df_fb[df_fb["page_accountType"]=="facebook_group"].copy()
fb_group_nodes = df_fb_group[['page_account_id','page_name','page_url','page_id_url','page_profileImage','page_subscriberCount','page_id','page_accountType','page_verified']]
fb_group_nodes = fb_group_nodes.rename(columns={'page_account_id': 'groupId',
                                              'page_name': 'facebookGroupName',
                                              'page_url': 'groupUrl',
                                              'page_id_url': 'groupIdUrl',
                                              'page_profileImage': 'profileImage',
                                              'page_subscriberCount': 'subscriberCount',
                                              'page_id': 'pageId',
                                              'page_accountType': 'accountType',
                                              'page_verified': 'verified'})
fb_group_nodes[":LABEL"] = "FacebookGroup"
fb_group_nodes = fb_group_nodes.rename(columns={'groupId': 'FacebookGroup:ID', 'facebookGroupName': 'facebookGroupName'})
fb_group_nodes = fb_group_nodes.drop_duplicates(subset='FacebookGroup:ID', keep="first")
fb_group_nodes["facebookGroupName"] = fb_group_nodes["facebookGroupName"].apply(lambda x : replace_double_quote(x))
fb_group_nodes["facebookGroupName"] = fb_group_nodes["facebookGroupName"].apply(lambda x : add_quote(x))
fb_group_nodes.to_csv('opt/csv/ct_tenet/fb_group_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_group_nodes.shape[0])
fb_group_nodes.sample(1)

245


,FacebookGroup:ID,facebookGroupName,groupUrl,groupIdUrl,profileImage,subscriberCount,pageId,accountType,verified,:LABEL
1938,8687066,"""台灣加油站！辣台妹加油讚！""",,https://www.facebook.com/groups/256642298342944,https://scontent-sea1-1.xx.fbcdn.net/v/t1.0-0/...,14168,256642298342944,facebook_group,False,FacebookGroup


<a id="create_facebook_profile_node"></a>
## Create Facebook profile Node

In [26]:
df_fb_profile = df_fb[df_fb["page_accountType"]=="facebook_profile"].copy()
fb_profile_nodes = df_fb_profile[['page_account_id','page_name','page_url','page_id_url','page_profileImage','page_subscriberCount','page_id','page_accountType','page_verified']]
fb_profile_nodes = fb_profile_nodes.rename(columns={'page_account_id': 'profileId',
                                              'page_name': 'facebookProfileName',
                                              'page_url': 'profileUrl',
                                              'page_id_url': 'profileIdUrl',
                                              'page_profileImage': 'profileImage',
                                              'page_subscriberCount': 'subscriberCount',
                                              'page_id': 'accountId',
                                              'page_accountType': 'accountType',
                                              'page_verified': 'verified'})
fb_profile_nodes[":LABEL"] = "FacebookPublicUser"
fb_profile_nodes = fb_profile_nodes.rename(columns={'profileId': 'FacebookPublicUser:ID', 'facebookProfileName': 'facebookProfileName'})
fb_profile_nodes = fb_profile_nodes.drop_duplicates(subset='FacebookPublicUser:ID', keep="first")
fb_profile_nodes["facebookProfileName"] = fb_profile_nodes["facebookProfileName"].apply(lambda x : replace_double_quote(x))
fb_profile_nodes["facebookProfileName"] = fb_profile_nodes["facebookProfileName"].apply(lambda x : add_quote(x))
fb_profile_nodes.to_csv('opt/csv/ct_tenet/fb_profile_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_profile_nodes.shape[0])
fb_profile_nodes.sample(1)

16


,FacebookPublicUser:ID,facebookProfileName,profileUrl,profileIdUrl,profileImage,subscriberCount,accountId,accountType,verified,:LABEL
0,3420777,"""Gregory Wong""",https://www.facebook.com/narcissistyoyo,https://facebook.com/narcissistyoyo,https://scontent-sea1-1.xx.fbcdn.net/v/t1.0-1/...,26334,,facebook_profile,True,FacebookPublicUser


<a id="create_facebook_expandedLink_node"></a>
## Create Facebook ExpandedLink Node

In [27]:
expanded_links = []
df_expanded_links = df_fb[df_fb["expandedLinks"]!=""]
for index, row in df_expanded_links.iterrows():
    for expandedLinks in row["expandedLinks"]:
        expanded_links.append({'ExpandedLink:ID':expandedLinks['expanded'].split(',')[0], 'name':expandedLinks['expanded'].split(',')[0], 'statusId':row['status_id'], ':LABEL':'ExpandedLink'})

fb_expanded_nodes = pd.DataFrame(expanded_links)
fb_expanded_nodes = fb_expanded_nodes.drop_duplicates(subset='ExpandedLink:ID', keep="first")
fb_expanded_nodes.to_csv('opt/csv/ct_tenet/fb_expanded_nodes.csv', index=False, encoding='utf-8')
print(fb_expanded_nodes.shape[0])
fb_expanded_nodes.head(1)

3216


,ExpandedLink:ID,name,statusId,:LABEL
0,https://www.facebook.com/PokeguideHK/photos/a....,https://www.facebook.com/PokeguideHK/photos/a....,10164803303730179,ExpandedLink


<a id="relationship_fbpage_fbpost"></a>
## Relationship of fb page POSTS fb post

In [28]:
to_id_list = list(fb_page_nodes['FacebookFanPage:ID'])
fb_page_relation = fb_post_nodes[['fanpageId','fanpageName','pageUrl','postUrl','FacebookPost:ID']]
fb_page_relation = fb_page_relation.rename(columns={'fanpageId': ':START_ID',
                                                    'FacebookPost:ID': ':END_ID'})
fb_page_relation['in_list'] = fb_page_relation[':START_ID'].apply(lambda x:if_in_list(int(x), to_id_list))
fb_page_relation = fb_page_relation[fb_page_relation['in_list']==1]
fb_page_relation["fanpageName"] = fb_page_relation["fanpageName"].apply(lambda x : replace_double_quote(x))
fb_page_relation["fanpageName"] = fb_page_relation["fanpageName"].apply(lambda x : add_quote(x))
del fb_page_relation['in_list']
fb_page_relation[':TYPE'] = 'POSTS'
fb_page_relation.to_csv('opt/csv/ct_tenet/fb_page_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_page_relation.shape[0])
fb_page_relation.sample(1)

1928


,:START_ID,fanpageName,pageUrl,postUrl,:END_ID,:TYPE
857,623395,"""'哈潑時尚 Harper's Bazaar Taiwan'""",https://www.facebook.com/BAZAAR.TW,https://www.facebook.com/BAZAAR.TW/posts/10157...,10157248328030964,POSTS


<a id="relationship_fbgroup_fbpost"></a>
## Relationship of fb group POSTS fb post

In [29]:
to_id_list = list(fb_group_nodes['FacebookGroup:ID'])
fb_group_relation = fb_post_nodes[['fanpageId','fanpageName','pageIdUrl','postUrl','FacebookPost:ID']]
fb_group_relation = fb_group_relation.rename(columns={'fanpageId': ':START_ID',
                                                      'pageIdUrl': 'profileIdUrl',
                                                      'fanpageName':'facebookGroupName',
                                                      'FacebookPost:ID': ':END_ID'})
fb_group_relation['in_list'] = fb_group_relation[':START_ID'].apply(lambda x:if_in_list(x, to_id_list))
fb_group_relation = fb_group_relation[fb_group_relation['in_list']==1]
fb_group_relation["facebookGroupName"] = fb_group_relation["facebookGroupName"].apply(lambda x : replace_double_quote(x))
fb_group_relation["facebookGroupName"] = fb_group_relation["facebookGroupName"].apply(lambda x : add_quote(x))
del fb_group_relation['in_list']
fb_group_relation[':TYPE'] = 'POSTS'
fb_group_relation.to_csv('opt/csv/ct_tenet/fb_group_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_group_relation.shape[0])
fb_group_relation.sample(1)

645


,:START_ID,facebookGroupName,profileIdUrl,postUrl,:END_ID,:TYPE
294,10824616,"""'Medium中文寫作者聯誼會'""",https://www.facebook.com/groups/404873543209905,https://www.facebook.com/groups/40487354320990...,1241115449585706,POSTS


<a id="relationship_fbprofile_fbpost"></a>
## Relationship of fb profile POSTS fb post

In [30]:
to_id_list = list(fb_profile_nodes['FacebookPublicUser:ID'])
fb_profile_relation = fb_post_nodes[['fanpageId','fanpageName','pageIdUrl','postUrl','FacebookPost:ID']]
fb_profile_relation = fb_profile_relation.rename(columns={'fanpageId': ':START_ID',
                                                          'pageIdUrl': 'groupIdUrl',
                                                          'fanpageName':'facebookProfileName',
                                                          'FacebookPost:ID': ':END_ID'})
fb_profile_relation['in_list'] = fb_profile_relation[':START_ID'].apply(lambda x:if_in_list(x, to_id_list))
fb_profile_relation = fb_profile_relation[fb_profile_relation['in_list']==1]
fb_profile_relation["facebookProfileName"] = fb_profile_relation["facebookProfileName"].apply(lambda x : replace_double_quote(x))
fb_profile_relation["facebookProfileName"] = fb_profile_relation["facebookProfileName"].apply(lambda x : add_quote(x))
del fb_profile_relation['in_list']
fb_profile_relation[':TYPE'] = 'POSTS'
fb_profile_relation.to_csv('opt/csv/ct_tenet/fb_profile_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_profile_relation.shape[0])
fb_profile_relation.sample(1)

21


,:START_ID,facebookProfileName,groupIdUrl,postUrl,:END_ID,:TYPE
2184,3419449,"""'張語昕'""",https://facebook.com/pinkyyyy520,https://www.facebook.com/pinkyyyy520/posts/194...,1941695475966456,POSTS


<a id="relationship_fb_expandedLink_posts"></a>
## Relationship of fb ExpandedLink and POSTS

In [31]:
to_id_list = list(fb_post_nodes['FacebookPost:ID'])
fb_expanded_relation = fb_expanded_nodes[['ExpandedLink:ID','name','statusId']]
fb_expanded_relation = fb_expanded_relation.rename(columns={'ExpandedLink:ID': ':START_ID',
                                                            'statusId': ':END_ID'})
fb_expanded_relation['in_list'] = fb_expanded_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
fb_expanded_relation.sample(5)
fb_expanded_relation = fb_expanded_relation[fb_expanded_relation['in_list']==1]
del fb_expanded_relation['in_list']
fb_expanded_relation[':TYPE'] = 'EXPANDS'
fb_expanded_relation.to_csv('opt/csv/ct_tenet/fb_expands_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_expanded_relation.shape[0])
fb_expanded_relation.sample(1)

3216


,:START_ID,name,:END_ID,:TYPE
3157,https://reurl.cc/AqZoRQ,https://reurl.cc/AqZoRQ,3088923147902692,EXPANDS


<a id="relationship_keywords_and_post"></a>
## Relationship of FacebookPost keywords and post

In [32]:
to_id_list = list(fb_post_nodes['FacebookPost:ID'])
fb_post_nodes.head()
fb_keyword_list = []
for index, row in df_fb[df_fb['keywords']!=''].iterrows():
    for keyword in row['keywords'].replace('\n',' ').split('/'):
        fb_keyword_list.append({"name":keyword, "postId":row['status_id']})

fb_keyword_nodes = pd.DataFrame(fb_keyword_list)
fb_keyword_relation = fb_keyword_nodes[['name','postId']]
fb_keyword_relation = fb_keyword_relation.rename(columns={'name': 'name',
                                                          'postId': 'to_id'})
fb_keyword_relation[":START_ID"] =  fb_keyword_relation["name"].map(keyword_id_map)
fb_keyword_relation["keyword"] = fb_keyword_relation["name"]
if 'name' in fb_keyword_relation.columns:
    del fb_keyword_relation['name']
fb_keyword_relation['in_list'] = fb_keyword_relation['to_id'].apply(lambda x:if_in_list(x, to_id_list))
fb_keyword_relation = fb_keyword_relation[fb_keyword_relation['in_list']==1]
del fb_keyword_relation['in_list']
fb_keyword_relation[':END_ID'] = fb_keyword_relation['to_id']
if 'to_id' in fb_keyword_relation.columns:
    del fb_keyword_relation['to_id']
fb_keyword_relation[':TYPE'] = 'RELATED_TO'
fb_keyword_relation = fb_keyword_relation[~fb_keyword_relation[':START_ID'].isnull()]
fb_keyword_relation["keyword"] = fb_keyword_relation["keyword"].apply(lambda x : replace_double_quote(x))
fb_keyword_relation["keyword"] = fb_keyword_relation["keyword"].apply(lambda x : add_quote(x))
fb_keyword_relation.to_csv('opt/csv/ct_tenet/fb_post_keyword_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_keyword_relation.shape[0])
fb_keyword_relation.sample(1)

63502


,:START_ID,keyword,:END_ID,:TYPE
62919,K100007858,"""7410937820""",4499143393444209,RELATED_TO


<a id="Relationship_facebookPost_hashtag_post"></a>
## Relationship of FacebookPost hashtag and post

In [42]:
to_id_list = list(fb_post_nodes['FacebookPost:ID'])
fb_hashtag_list = []
for index, row in df_fb[df_fb['hashtags']!=''].iterrows():
    for hashtag in row['hashtags'].replace('\n',' ').strip().split('/'):
        fb_hashtag_list.append({"hashtag":hashtag, "postId":row['status_id']})

if len(fb_hashtag_list)>0:
    fb_post_hashtag_nodes = pd.DataFrame(fb_hashtag_list)
    fb_hashtag_relation = fb_post_hashtag_nodes[['hashtag','postId']]
    fb_hashtag_relation = fb_hashtag_relation.rename(columns={'hashtag': 'hashtag',
                                                                'postId': ':END_ID'})
    fb_hashtag_relation[":START_ID"] =  fb_hashtag_relation["hashtag"].map(hashtags_id_map)
    fb_hashtag_relation[":END_ID"] = fb_hashtag_relation[":END_ID"].apply(lambda x:str(x))
    fb_hashtag_relation['in_list'] = fb_hashtag_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
    fb_hashtag_relation = fb_hashtag_relation[fb_hashtag_relation['in_list']==1]
    del fb_hashtag_relation['in_list']
    fb_hashtag_relation[':TYPE'] = 'TAGS'
    fb_hashtag_relation["hashtag"] = fb_hashtag_relation["hashtag"].apply(lambda x : replace_double_quote(x))
    fb_hashtag_relation["hashtag"] = fb_hashtag_relation["hashtag"].apply(lambda x : add_quote(x))
    fb_hashtag_relation = fb_hashtag_relation[[":START_ID","hashtag",":END_ID",":TYPE"]]
    fb_hashtag_relation = fb_hashtag_relation[~fb_hashtag_relation[':START_ID'].isnull()] 
    fb_hashtag_relation.to_csv('opt/csv/ct_tenet/fb_hashtag_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
    print(fb_hashtag_relation.shape[0])
else:
    fb_hashtag_relation = pd.DataFrame(columns=[":START_ID","hashtag",":END_ID",":TYPE"])
    fb_hashtag_relation.to_csv('opt/csv/ct_tenet/fb_hashtag_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
fb_hashtag_relation.head(1)

6856


,:START_ID,hashtag,:END_ID,:TYPE
0,H100000001,"""#01周報""",1687588074752781,TAGS


<a id="relationship_mention_and_post"></a>
## Relationship of Facebook mention and post

In [33]:
to_id_list = list(fb_post_nodes['FacebookPost:ID'])
fb_post_nodes.head()
fb_mention_list = []
for index, row in df_fb[df_fb['mention']!=''].iterrows():
    for mention in row['mention'].replace('\n',' ').split('/'):
        if len(mention.strip())>0:
            if ' ' in mention:
                mention = mention.split(' ')[0]
            if mention[-1:]=='.':
                mention  = mention[:-1]
            if len(mention)>1:
                fb_mention_list.append({"name":mention, "postId":row['status_id']})
fb_post_mention_nodes = pd.DataFrame(fb_mention_list)
fb_mention_relation = fb_post_mention_nodes[['name','postId']]
fb_mention_relation = fb_mention_relation.rename(columns={'name': 'name',
                                                          'postId': ':END_ID'})
fb_mention_relation['in_list'] = fb_mention_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
fb_mention_relation = fb_mention_relation[fb_mention_relation['in_list']==1]
del fb_mention_relation['in_list']
fb_mention_relation[':TYPE'] = 'MENTIONS'
fb_mention_relation[":START_ID"] =  fb_mention_relation["name"].map(mentions_id_map)
fb_mention_relation = fb_mention_relation[[':START_ID'] + fb_mention_relation.columns[:-1].tolist()]
fb_mention_relation["name"] = fb_mention_relation["name"].apply(lambda x : replace_double_quote(x))
fb_mention_relation["name"] = fb_mention_relation["name"].apply(lambda x : add_quote(x))
fb_mention_relation = fb_mention_relation[fb_mention_relation["name"] != '"#"']
fb_mention_relation = fb_mention_relation[fb_mention_relation["name"] != '"@"']
fb_mention_relation = fb_mention_relation[fb_mention_relation[':START_ID'].notnull()]
fb_mention_relation.to_csv('opt/csv/ct_tenet/fb_mention_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(fb_mention_relation.shape[0])
fb_mention_relation.sample(1)

139


,:START_ID,name,:END_ID,:TYPE
135,M100000086,"""@國賓大戲院""",1895913057210643,MENTIONS


<a id="create_instagram_post_node"></a>
## Create Instagram post Node

In [34]:
print(df_ig.shape[0])
df_ig.sample(1)

209


,platformId,platform,date,updated,type,description,message,expandedLinks,link,postUrl,...,page_id,page_accountType,page_verified,context,shortCode,postId,hashtags,mention,userName,keywords
2783,2383467857630220898,Instagram,2020-08-25 10:32:11,2020-10-23 00:37:24,photo,-\n｜Wazaiii 時尚郊遊去｜李英宏：時間就是青春，與 HAMILTON 漢米爾頓共賞...,,,,https://www.instagram.com/p/CETx2NrnFJi/,...,4525330842,,False,-\n｜Wazaiii 時尚郊遊去｜李英宏：時間就是青春，與 HAMILTON 漢米爾頓共賞...,CETx2NrnFJi,2383467857630220898,#Wazaiii/#Jewelry/#Hamilton/#KhakiNavyBeLOWZER...,@wazaiii/@好友搜尋/@wazaiii\n#Wazaiii,wazaiii,戴上/輕量/時計/BeLOWZERO/一只/上山下海/HAMILTON/外型/最早/技術/S...


In [35]:
ig_post_nodes = df_ig[['postId','shortCode','description','date','actual_likeCount','actual_commentCount','postUrl','page_id','page_name','userName','page_url']].copy()
ig_post_nodes = ig_post_nodes.rename(columns={'postId': 'postId',
                                              'shortCode': 'shortCode',
                                              'description': 'text',
                                              'date': 'created',
                                              'actual_likeCount': 'likeCount',
                                              'actual_commentCount': 'commentCount',
                                              'postUrl': 'postUrl',
                                              'page_id': 'igId',
                                              'page_name': 'igName',
                                              'userName': 'userName',
                                              'page_url': 'pageUrl'})
ig_post_nodes[":LABEL"] = "InstagramPost"
ig_post_nodes = ig_post_nodes.rename(columns={'postId': 'InstagramPost:ID', 'text': 'text'})
ig_post_nodes = ig_post_nodes.drop_duplicates(subset='InstagramPost:ID', keep="first")
ig_post_nodes["text"] = ig_post_nodes["text"].apply(lambda x : replace_double_quote(x))
ig_post_nodes["text"] = ig_post_nodes["text"].apply(lambda x : add_quote(x))
ig_post_nodes["igName"] = ig_post_nodes["igName"].apply(lambda x : replace_double_quote(x))
ig_post_nodes["igName"] = ig_post_nodes["igName"].apply(lambda x : add_quote(x))
ig_post_nodes.to_csv('opt/csv/ct_tenet/ig_post_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(ig_post_nodes.shape[0])
ig_post_nodes.head(1)

209


,InstagramPost:ID,shortCode,text,created,likeCount,commentCount,postUrl,igId,igName,userName,pageUrl,:LABEL
2594,2406832171454085405,CFmyRulgLkd,"""【#不是影評／《天能》與《闇》：自大狂與強迫症的燒腦對讀（下）】 . 以今日的科幻作品而言...",2020-09-26 16:12:54,0,0,https://www.instagram.com/p/CFmyRulgLkd/,42977279,"""紅眼 redeye777""",redeye777,https://www.instagram.com/redeye777/,InstagramPost


<a id="create_instagram_user_node"></a>
## Create Instagram user Node

In [36]:
ig_user_nodes = df_ig[['page_id','page_name','userName','page_url','page_profileImage','page_subscriberCount','page_verified']].copy()
ig_user_nodes = ig_user_nodes.rename(columns={'page_id': 'igId',
                                              'page_name': 'pageName',
                                              'userName': 'igName',
                                              'page_url': 'pageUrl',
                                              'page_profileImage': 'profileImage',
                                              'page_subscriberCount': 'followerCount',
                                              'page_verified': 'verified'})
ig_user_nodes["userName"] = ig_user_nodes["igName"]
ig_user_nodes["igName"] = ig_user_nodes["pageName"]
if 'pageName' in ig_user_nodes.columns:
    del ig_user_nodes['pageName']
ig_user_nodes[":LABEL"] = "InstagramUser"
ig_user_nodes = ig_user_nodes.rename(columns={'igId': 'InstagramUser:ID', 'userName': 'userName'})
ig_user_nodes = ig_user_nodes.drop_duplicates(subset='InstagramUser:ID', keep="first")
ig_user_nodes["igName"] = ig_user_nodes["igName"].apply(lambda x : replace_double_quote(x))
ig_user_nodes["igName"] = ig_user_nodes["igName"].apply(lambda x : add_quote(x))
ig_user_nodes.to_csv('opt/csv/ct_tenet/ig_user_nodes.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(ig_user_nodes.shape[0])
ig_user_nodes.head(1)

97


,InstagramUser:ID,igName,pageUrl,profileImage,followerCount,verified,userName,:LABEL
2594,42977279,"""紅眼 redeye777""",https://www.instagram.com/redeye777/,https://scontent-sea1-1.cdninstagram.com/v/t51...,1427,False,redeye777,InstagramUser


<a id="relationship_ig_user_post"></a>
## Relationship of Instagram user and post

In [37]:
to_id_list = list(ig_post_nodes['InstagramPost:ID'])
ig_user_post_relation = ig_post_nodes[['igId','igName','userName','InstagramPost:ID']]
ig_user_post_relation = ig_user_post_relation.rename(columns={'igId': ':START_ID',
                                                              'InstagramPost:ID': ':END_ID'})
ig_user_post_relation['in_list'] = ig_user_post_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
ig_user_post_relation = ig_user_post_relation[ig_user_post_relation['in_list']==1]
del ig_user_post_relation['in_list']
ig_user_post_relation[':TYPE'] = 'POSTS'
ig_user_post_relation.to_csv('opt/csv/ct_tenet/ig_user_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(ig_user_post_relation.shape[0])
ig_user_post_relation.sample(1)

209


,:START_ID,igName,userName,:END_ID,:TYPE
2767,1798332793,"""華納兄弟台灣粉絲俱樂部""",wbtw.fanclub,2385628837759971092,POSTS


<a id="relationship_ig_keywords_post"></a>
## Relationship of Instagram keywords and post

In [38]:
to_id_list = list(ig_post_nodes['InstagramPost:ID'])
ig_post_nodes.head()
ig_keyword_list = []
for index, row in df_ig[df_ig['keywords']!=''].iterrows():
    for keyword in row['keywords'].replace('\n',' ').split('/'):
        ig_keyword_list.append({"name":keyword, "postId":row['postId']})

ig_keyword_nodes = pd.DataFrame(ig_keyword_list)
ig_keyword_relation = ig_keyword_nodes[['name','postId']]
ig_keyword_relation = ig_keyword_relation.rename(columns={'name': 'name',
                                                          'postId': 'to_id'})
ig_keyword_relation[":START_ID"] =  ig_keyword_relation["name"].map(keyword_id_map)
ig_keyword_relation["keyword"] = ig_keyword_relation["name"]
if 'name' in ig_keyword_relation.columns:
    del ig_keyword_relation['name']
ig_keyword_relation['in_list'] = ig_keyword_relation['to_id'].apply(lambda x:if_in_list(x, to_id_list))
ig_keyword_relation = ig_keyword_relation[ig_keyword_relation['in_list']==1]
del ig_keyword_relation['in_list']
ig_keyword_relation[':END_ID'] = ig_keyword_relation['to_id']
if 'to_id' in ig_keyword_relation.columns:
    del ig_keyword_relation['to_id']
ig_keyword_relation[':TYPE'] = 'RELATED_TO'
ig_keyword_relation["keyword"] = ig_keyword_relation["keyword"].apply(lambda x : replace_double_quote(x))
ig_keyword_relation["keyword"] = ig_keyword_relation["keyword"].apply(lambda x : add_quote(x))
ig_keyword_relation.to_csv('opt/csv/ct_tenet/ig_post_keyword_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(ig_keyword_relation.shape[0])
ig_keyword_relation.sample(1)

7083


,:START_ID,keyword,:END_ID,:TYPE
6294,K100015706,"""俐落""",2383468013482237395,RELATED_TO


<a id="relationship_ig_hashtag_post"></a>
## Relationship of Instagram hashtag and post

In [39]:
to_id_list = list(ig_post_nodes['InstagramPost:ID'])
ig_hashtag_list = []
for index, row in df_ig[df_ig['hashtags']!=''].iterrows():
    for hashtag in row['hashtags'].replace('\n',' ').split('/'):
        if len(hashtag.strip())>0:
            if ' ' in hashtag:
                hashtag = hashtag.split(' ')[0]
            if hashtag[-1:]=='.':
                hashtag  = hashtag[:-1]
            ig_hashtag_list.append({"name":hashtag, "postId":row['postId']})

ig_post_hashtag_nodes = pd.DataFrame(ig_hashtag_list)
ig_hashtag_relation = ig_post_hashtag_nodes[['name','postId']]
ig_hashtag_relation = ig_hashtag_relation.rename(columns={'name': 'name',
                                                          'postId': ':END_ID'})
ig_hashtag_relation['in_list'] = ig_hashtag_relation[':END_ID'].apply(lambda x:if_in_list(x, to_id_list))
ig_hashtag_relation = ig_hashtag_relation[ig_hashtag_relation['in_list']==1]
del ig_hashtag_relation['in_list']
ig_hashtag_relation[':TYPE'] = 'TAGS'
ig_hashtag_relation[":START_ID"] =  ig_hashtag_relation["name"].map(hashtags_id_map)
ig_hashtag_relation = ig_hashtag_relation[[':START_ID'] + ig_hashtag_relation.columns[:-1].tolist()]
ig_hashtag_relation["name"] = ig_hashtag_relation["name"].apply(lambda x : replace_double_quote(x))
ig_hashtag_relation["name"] = ig_hashtag_relation["name"].apply(lambda x : add_quote(x))
ig_hashtag_relation = ig_hashtag_relation[ig_hashtag_relation["name"] != '"#"']
ig_hashtag_relation = ig_hashtag_relation[ig_hashtag_relation[':START_ID'].notnull()]
ig_hashtag_relation.to_csv('opt/csv/ct_tenet/ig_hashtag_post_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(ig_hashtag_relation.shape[0])
ig_hashtag_relation.sample(1)

1692


,:START_ID,name,:END_ID,:TYPE
974,H100000213,"""#香港電影""",2394952780467896127,TAGS


<a id="relationship_ig_mention_post"></a>
## Relationship of Instagram mention and post

In [40]:
ig_with_mention = df_ig[df_ig['mention'].str.len() > 0]
ig_with_mention.sample(1)

,platformId,platform,date,updated,type,description,message,expandedLinks,link,postUrl,...,page_id,page_accountType,page_verified,context,shortCode,postId,hashtags,mention,userName,keywords
2776,2384788667732070001,Instagram,2020-08-27 06:16:24,2020-09-21 22:10:05,photo,（無雷觀後感） @kristinswaltz \n《天能TENET》Christopher ...,,,,https://www.instagram.com/p/CEYeKh8jA5x/,...,1798332793,,True,（無雷觀後感） @kristinswaltz \n《天能TENET》Christopher ...,CEYeKh8jA5x,2384788667732070001,#天能/#天能Tenet/#tenet/#tenetmovie/#christopherno...,@kristinswaltz,wbtw.fanclub,全神貫注/IMAX/二十九/愛上/刻意/衝擊/唯一/畫面/鏡頭/流動/口氣/生成/Nolan...


In [41]:
to_id_list = list(ig_user_nodes['userName'])
mention_not_in_user_list = []
ig_post_mention_relation_list = []
df_ig_post_mention = ig_with_mention[['postId','postUrl','mention']]
for index, row in df_ig_post_mention.iterrows():
    for mention_user in row['mention'].split('/'):
        userName = mention_user.split('\n')[0].strip().replace('@','')
        if userName in to_id_list:
            mention_userName = list(ig_user_nodes[ig_user_nodes['userName']==userName]['userName'])[0]
            mention_igName = list(ig_user_nodes[ig_user_nodes['userName']==userName]['igName'])[0]
            END_ID = list(ig_user_nodes[ig_user_nodes['userName']==userName]['InstagramUser:ID'])[0]
            ig_post_mention_relation_list.append({":START_ID":row['postId'], "igName":mention_igName, "userName":mention_userName, ":END_ID":END_ID, ":TYPE":"MENTIONS"})
        else:
            mention_not_in_user_list.append(userName)
ig_post_mention_relation = pd.DataFrame(ig_post_mention_relation_list)
ig_post_mention_relation.to_csv('opt/csv/ct_tenet/ig_post_mention_rel.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')
print(ig_post_mention_relation.shape[0])
ig_post_mention_relation.sample(1)

30


,:START_ID,igName,userName,:END_ID,:TYPE
22,2383468013482237395,"""Wazaiii""",wazaiii,4525330842,MENTIONS


<a id="import_neo4j"></a>
## Import to Neo4j

```
.\bin\neo4j-admin import --nodes=import/fb_expanded_nodes.csv --nodes=import/fb_group_nodes.csv --nodes=import/fb_page_nodes.csv --nodes=import/fb_post_nodes.csv --nodes=import/fb_profile_nodes.csv --nodes=import/ig_post_nodes.csv --nodes=import/ig_user_nodes.csv --nodes=import/post_hashtags_nodes.csv --nodes=import/post_keyword_nodes.csv --nodes=import/post_mention_nodes.csv --relationships=import/fb_expands_rel.csv --relationships=import/fb_group_post_rel.csv --relationships=import/fb_hashtag_post_rel.csv --relationships=import/fb_mention_post_rel.csv --relationships=import/fb_page_post_rel.csv --relationships=import/fb_post_keyword_rel.csv --relationships=import/fb_profile_post_rel.csv --relationships=import/ig_hashtag_post_rel.csv --relationships=import/ig_post_keyword_rel.csv --relationships=import/ig_post_mention_rel.csv --relationships=import/ig_user_post_rel.csv 
```